In [104]:
import cv2
import math

In [105]:
cv2.data.haarcascades

'/Users/bekkblando/.local/share/virtualenvs/assignment5-mQdkBZfd/lib/python3.7/site-packages/cv2/data/'

In [113]:
    
class Faces():
    
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier('classifiers/frontalFace10/haarcascade_frontalface_alt2.xml')
        self.left_eye_cascade = cv2.CascadeClassifier(f'{cv2.data.haarcascades}/haarcascade_lefteye_2splits.xml')
        self.right_eye_cascade = cv2.CascadeClassifier(f'{cv2.data.haarcascades}/haarcascade_righteye_2splits.xml')

        if cv2.__version__ == '4.0.0' or cv2.__version__ == '4.2.0':
            self.noseCascadeName = './classifiers/haarcascade_mcs_nose.xml'
        elif cv2.__version__ == '3.4.1':
            self.noseCascadeName = '/Nariz.xml'

        if cv2.__version__ == '4.0.0' or cv2.__version__ == '4.2.0':
            self.mouthCascadeName = '/haarcascade_smile.xml'
        elif cv2.__version__ == '3.4.1':
            self.mouthCascadeName = '/Mouth.xml'
            
        self.nose_cascade = cv2.CascadeClassifier(self.noseCascadeName)
        self.mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + self.mouthCascadeName)
        
    def draw_cascades(self, items, frame, label):
        for (x,y,w,h) in items:
            cv2.rectangle(frame,(x, y),(x + w, y + h),(0,255,0),2)
            cv2.putText(frame,label,(x + x, y + y), 1, 1, (0, 255, 0), 1)
          
    def in_face(self, obj):
        for (x,y,w,h) in self.faces:
            if(obj[0] >= x and obj[1] >= y):
                return True
        return False
    
    def remove_not_in_face(self, objs):
        return list(filter(self.in_face, objs))
    

    def clean(self):
        features = [self.left_eyes, self.right_eyes, self.mouths, self.noses, self.faces]
        return features
    
    def get_faces(self, frame):
        return self.face_cascade.detectMultiScale(frame, 1.1, 5)
    
    def get_left_eyes(self, frame):
        return self.left_eye_cascade.detectMultiScale(frame, 1.1, 5)
    
    def get_right_eyes(self, frame):
        return self.right_eye_cascade.detectMultiScale(frame, 1.1, 5)
    
    def get_mouths(self, frame):
        return self.mouth_cascade.detectMultiScale(frame, 1.1, 5)
    
    def get_noses(self, frame):
        return self.nose_cascade.detectMultiScale(frame, 1.1, 5)
    
    
    def read_clean_view(self, gray, frame):
        self.faces = []
        self.left_eyes = []
        self.right_eyes = []
        self.mouths = []
        self.noses = []
        
        # Get those rectangles around those faces
        self.faces = self.get_faces(gray)
        self.draw_cascades(self.faces, frame, "Faces")
        
        for (x, y, w, h) in self.faces:
            filtered_grey = gray[y: y + h, x: x + w]
            color = frame[y: y + h, x: x + w]

            self.draw_cascades(self.get_left_eyes(filtered_grey), color, "Left Eyes")
            # Draw a rectangle around the left eyes
           

            # Draw a rectangle around the right eyes
            self.draw_cascades(self.get_right_eyes(filtered_grey), color, "Right Eyes")
            

            # Draw a rectangle around the mouth
            self.draw_cascades(self.get_mouths(filtered_grey), color, "Mouths")


            # Draw a rectangle around the nose
            self.draw_cascades(self.get_noses(filtered_grey), color, "Noses")

        # Clean
        # self.clean()
        
        # Draw
        # self.draw(frame)
    
face = Faces()     

In [114]:
cap = cv2.VideoCapture(0)

In [115]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Draw a rectangle around the faces
    face.read_clean_view(gray, frame)
    
    # Display Frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

1 1.0
2 1.0


IndexError: list index out of range

In [99]:
cap.release()
cv2.destroyAllWindows()